In [1]:
import pandas as pd
import numpy as np

In [2]:
# creating lists to use for later looping
years = [2013,2014, 2015,2016,2017,2018,2019]
leagues = ["english_premier_league","italian_serie_a","spanish_primera_division"]

In [3]:
# Creating a function that will merge the various years for each league chosen
def data_org(league):
    appended_data = []
    for year in years:
        file_path=f"../raw/data/{year}/{league}.csv"
        df_league = pd.read_csv(file_path)
        appended_data.append(df_league)
    new_df = pd.concat(appended_data)
    new_df = new_df[["year","club_name","transfer_movement","fee_cleaned"]]
    new_df = new_df[new_df["transfer_movement"]=='in']
    return new_df

In [4]:
# Applying the function to build our datasets
premier_league_df = data_org(leagues[0])
italian_df = data_org(leagues[1])
spanish_df = data_org(leagues[2])

In [5]:
# Creating a function that formats the dataframes to suit requirements for analysis
def group_and_clean(df):
    df_group = df.groupby(["year","club_name"]).sum()
    df_group = df_group.sort_values(by=["year","fee_cleaned"],ascending=False).rename(columns={"fee_cleaned": "Transfer Spend (Euros)"}).reset_index()
    df_group = df_group.rename(columns={"club_name":"Teams"})
    return df_group

In [6]:
# Applying the function to format our datasets
italian_transfers = group_and_clean(italian_df)
english_transfers = group_and_clean(premier_league_df)
spanish_transfers = group_and_clean(spanish_df)

In [7]:
# A function that will be used when calculating the moving average 
# In this data set, there are teams that have not been in the league for each year we are working with due to promotion / relegation. 
# This function ensures that we are only calculating the Moving Average for teams that have been there for consecutive years.

def year_test(time):
    count = 0
    length = len(time) - 1
    outcome = "False"
    # Using the difference between years to determine if they are sequential.
    for year1, year2 in zip(time,time[1:]):
        total = year2 - year1
        count += total
    if length == count:
        outcome = "True"
    return outcome

In [8]:
# Function to calculate the moving average (MA)
def moving_average(df):
    # empty list to store our dataframes
    append_team = []
    
    # sorting values by Team and then year to allow average to calculate
    df = df.sort_values(by=["Teams","year"], ascending = True)
    
    # generating a unique list of the teams
    teams = df["Teams"].unique().tolist()

    for team in teams:
        # filtering on an individual team to create their 3yr MA
        team_df = df[df["Teams"]==team]
        years = team_df["year"].tolist()
        if year_test(years) == "True":
            # using 'rolling' function to create MA
            team_df["MA"] = team_df.iloc[:,2].rolling(window=3,min_periods = 3).mean().round(2)
            team_df = team_df[["year","Teams","MA"]]
            append_team.append(team_df)
    team_list = pd.concat(append_team)
    
    #joining the dataframes
    team_join = pd.merge(df,team_list,how = "outer",on=["year","Teams"])
    return team_join

In [9]:
# Applying MA function to each league
spanish_summary = moving_average(spanish_transfers)
english_summary = moving_average(english_transfers)
italian_summary = moving_average(italian_transfers)

In [10]:
# Cleaning Italian Data
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s$","")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+FC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+BC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+CFC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?UC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?US", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?SSC\s?", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?","")

In [11]:
# Cleaning English Data
english_summary["Teams"] = english_summary["Teams"].str.replace("\s$","")
english_summary["Teams"] = english_summary["Teams"].str.replace("\s+FC", "")
english_summary["Teams"] = english_summary["Teams"].str.replace("\s+AFC", "")
english_summary["Teams"] = english_summary["Teams"].str.replace("^AFC\s", "")

In [12]:
# Outputting data to CSV
italian_summary.to_csv("../cleaned/italian_transfers.csv")
english_summary.to_csv("../cleaned/english_transfers.csv")
spanish_summary.to_csv("../cleaned/spanish_transfers.csv")